In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Recommendation_system').getOrCreate()

In [4]:
data = spark.read.json("Musical_Instruments_5.json")

In [5]:
data.show(5,truncate=True)

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|1384719342|  [0, 0]|    5.0|Not much to write...|02 28, 2014|A2IBPI20UZIR0U|cassandra tu "Yea...|                good|    1393545600|
|1384719342|[13, 14]|    5.0|The product does ...|03 16, 2013|A14VAT5EAX3D9S|                Jake|                Jake|    1363392000|
|1384719342|  [1, 1]|    5.0|The primary job o...|08 28, 2013|A195EZSQDW3E21|Rick Bennette "Ri...|It Does The Job Well|    1377648000|
|1384719342|  [0, 0]|    5.0|Nice windscreen p...|02 14, 2014|A2C00NNG1ZQQG2|RustyBill "Sunday...|GOOD WINDSCREEN F...|    1392336000|
|1384719342|  [0, 0]|    5.0|This pop filter i...|02 21

In [6]:
data_sub = data.select(['asin', 'overall', 'reviewerID'])

In [7]:
data_sub.count()

10261

In [8]:
from pyspark.sql.functions import col, udf
from pyspark.sql.functions import isnan, when, count, col

In [9]:
data_sub.show(5, truncate=True)

+----------+-------+--------------+
|      asin|overall|    reviewerID|
+----------+-------+--------------+
|1384719342|    5.0|A2IBPI20UZIR0U|
|1384719342|    5.0|A14VAT5EAX3D9S|
|1384719342|    5.0|A195EZSQDW3E21|
|1384719342|    5.0|A2C00NNG1ZQQG2|
|1384719342|    5.0| A94QU4C90B1AX|
+----------+-------+--------------+
only showing top 5 rows



In [10]:
data_sub.select([count(when(col(c).isNull(), c)).alias(c) for c in data_sub.columns]).toPandas().T

,0
asin,0
overall,0
reviewerID,0


In [11]:
# Distinct users and movies
users = data_sub.select("reviewerID").distinct().count()
products = data_sub.select("asin").distinct().count()
numerator = data_sub.count()

In [12]:
display(numerator, users, products)

10261

1429

900

In [13]:
# Number of ratings matrix could contain if no empty cells
denominator = users * products
denominator

1286100

In [14]:
#Calculating sparsity
sparsity = 1 - (numerator*1.0 / denominator)
print ("Sparsity: "), sparsity

Sparsity: 


(None, 0.992021615737501)

In [15]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [16]:
# Converting String to index
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col


In [17]:
# Create an indexer
indexer = StringIndexer(inputCol='asin',  outputCol='asin_idx')
# Indexer identifies categories in the data
indexer_model = indexer.fit(data_sub)
# Indexer creates a new column with numeric index values
data_indexed = indexer_model.transform(data_sub)
# Repeat the process for the other categorical feature
indexer1 = StringIndexer(inputCol='reviewerID', outputCol='reviewerID_idx')
indexer1_model = indexer1.fit(data_indexed)
data_indexed = indexer1_model.transform(data_indexed)

In [18]:
data_indexed.show(5, truncate=True)

+----------+-------+--------------+--------+--------------+
|      asin|overall|    reviewerID|asin_idx|reviewerID_idx|
+----------+-------+--------------+--------+--------------+
|1384719342|    5.0|A2IBPI20UZIR0U|   703.0|          66.0|
|1384719342|    5.0|A14VAT5EAX3D9S|   703.0|         266.0|
|1384719342|    5.0|A195EZSQDW3E21|   703.0|         395.0|
|1384719342|    5.0|A2C00NNG1ZQQG2|   703.0|        1048.0|
|1384719342|    5.0| A94QU4C90B1AX|   703.0|        1311.0|
+----------+-------+--------------+--------+--------------+
only showing top 5 rows



In [19]:
data_indexed.select([count(when(col(c).isNull(), c)).alias(c) for c in data_indexed.columns]).toPandas().T

,0
asin,0
overall,0
reviewerID,0
asin_idx,0
reviewerID_idx,0


In [20]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data_indexed.randomSplit([0.8, 0.2])

In [21]:
# Creating ALS model and fitting data
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [22]:
als = ALS(maxIter=5, 
          regParam=0.09, 
          rank = 25,
          userCol="reviewerID_idx", 
          itemCol="asin_idx", 
          ratingCol="overall", 
          coldStartStrategy="drop",
          nonnegative=True)
model = als.fit(training)

In [23]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [24]:
predictions.select(["asin_idx", "reviewerID_idx", 
                    "overall", "prediction"]).show(5)

+--------+--------------+-------+----------+
|asin_idx|reviewerID_idx|overall|prediction|
+--------+--------------+-------+----------+
|   323.0|         471.0|    3.0|  4.290072|
|     8.0|         148.0|    4.0| 2.6836226|
|    34.0|         496.0|    5.0| 4.5216136|
|   184.0|        1088.0|    4.0| 4.9800196|
|   243.0|         148.0|    3.0|  2.398322|
+--------+--------------+-------+----------+
only showing top 5 rows



In [25]:
evaluator = RegressionEvaluator(metricName="rmse", 
                                labelCol="overall",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.190920820931131


In [26]:
# On average, this model is ~ 1.2 from perfect recommendations.

### Providing Recommendations: for all users

In [27]:
# get 20 recommendations which have highest rating.
user_recs = model.recommendForAllUsers(20)

In [28]:
for user in user_recs.head(5):
    print(user)
    print("\n")

Row(reviewerID_idx=12, recommendations=[Row(asin_idx=870, rating=6.345491409301758), Row(asin_idx=579, rating=6.280359745025635), Row(asin_idx=290, rating=6.246193885803223), Row(asin_idx=801, rating=6.1191558837890625), Row(asin_idx=891, rating=6.10900354385376), Row(asin_idx=668, rating=6.108401298522949), Row(asin_idx=517, rating=6.089846611022949), Row(asin_idx=781, rating=6.069179534912109), Row(asin_idx=857, rating=6.052563190460205), Row(asin_idx=273, rating=6.043859958648682), Row(asin_idx=490, rating=6.040466785430908), Row(asin_idx=460, rating=6.007510662078857), Row(asin_idx=339, rating=5.978326320648193), Row(asin_idx=274, rating=5.975578784942627), Row(asin_idx=855, rating=5.968652248382568), Row(asin_idx=812, rating=5.968412399291992), Row(asin_idx=393, rating=5.9656805992126465), Row(asin_idx=596, rating=5.956305980682373), Row(asin_idx=539, rating=5.955963611602783), Row(asin_idx=886, rating=5.92951774597168)])


Row(reviewerID_idx=22, recommendations=[Row(asin_idx=618,

### Converting back to string form


In [29]:
import pandas as pd
recs=model.recommendForAllUsers(10).toPandas()
nrecs=recs.recommendations.apply(pd.Series) \
 .merge(recs, right_index = True, left_index = True) \
 .drop(["recommendations"], axis = 1) \
 .melt(id_vars = ['reviewerID_idx'], value_name = "recommendation") \
 .drop("variable", axis = 1) \
 .dropna() 
nrecs=nrecs.sort_values('reviewerID_idx')
nrecs=pd.concat([nrecs['recommendation'].apply(pd.Series), 
 nrecs['reviewerID_idx']], axis = 1)
nrecs.columns = [ 
 'ProductID_index',
 'Rating',
 'UserID_index' 
 ]

In [30]:
md=data_indexed.select(['reviewerID', 'reviewerID_idx', 'asin', 'asin_idx'])
md=md.toPandas()
dict1=dict(zip(md['reviewerID_idx'],md['reviewerID']))
dict2=dict(zip(md['asin_idx'],md['asin']))
nrecs['reviewerID']=nrecs['UserID_index'].map(dict1)
nrecs['asin']=nrecs['ProductID_index'].map(dict2)
nrecs=nrecs.sort_values('reviewerID')
nrecs.reset_index(drop=True, inplace=True)
new=nrecs[['reviewerID','asin','Rating']]
new['recommendations'] = list(zip(new.asin, new.Rating))
res=new[['reviewerID','recommendations']] 
res_new=res['recommendations'].groupby([res.reviewerID]).apply(list).reset_index()

C:\Users\AW\AppData\Local\Temp\ipykernel_30104\2091850030.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['recommendations'] = list(zip(new.asin, new.Rating))


In [31]:
res_new

,reviewerID,recommendations
0,A00625243BI8W1SSZNLMD,"[(B000RKAFIU, 5.466587543487549), (B001IM5KFY,..."
1,A10044ECXDUVKS,"[(B000RKAFIU, 5.0673322677612305), (B00923G9Q0..."
2,A102MU6ZC9H1N6,"[(B00923G9Q0, 5.872036933898926), (B000RWJQRE,..."
3,A109JTUZXO61UY,"[(B000W00X1Y, 6.402008056640625), (B00923G9Q0,..."
4,A109ME7C09HM2M,"[(B0064BFNME, 5.840763568878174), (B001E3BSKS,..."
...,...,...
1424,AZJPNK73JF3XP,"[(B001E3BSKS, 5.660131454467773), (B003S3S0DU,..."
1425,AZMHABTPXVLG3,"[(B003YVRGN6, 3.9164352416992188), (B000VJZOLK..."
1426,AZMIKIG4BB6BZ,"[(B009E3EWPI, 5.349851608276367), (B004MNTIL8,..."
1427,AZPDO6FLSMLFP,"[(B0001FTVD6, 5.070217132568359), (B0007WPCKE,..."
